<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/Test_BoNet_RSNA_RHPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet

fatal: destination path 'Bonet' already exists and is not an empty directory.


In [2]:
import numpy as np # linear algebra # เป็นการ import numpy เพื่อเปิดใช้ฟังก์ชันในการทำงาน
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) # เป็น library ในการจัดการ dataframe

In [3]:
import Bonet

In [4]:
# Standard lib imports
import os
import csv
import glob
import time
import argparse
import warnings
import pandas as pd
import os.path as osp

In [5]:
# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
!pip install horovod
import horovod.torch as hvd
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Other imports
from tqdm import tqdm
import pdb

In [7]:
warnings.filterwarnings("ignore")

In [8]:
parser = argparse.ArgumentParser()

In [9]:
# Dataloading-related settings
parser.add_argument('--heatmaps', default=False, action='store_true',
                help='Train model with gaussian heatmaps')
parser.add_argument('--cropped', default=False, action='store_true',
                help='Train model with cropped images according to bbox')
parser.add_argument('--dataset', default='RSNA', type=str,choices=['RSNA','RHPE'],
                help='Dataset to perform training')

parser.add_argument('--data-train', default='data/train/', type=str,
                help='path to train data folder')
parser.add_argument('--ann-path-train', default='train.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('--rois-path-train', default='train.json',
                type=str, help='path to ROIs annotations in coco format')

parser.add_argument('--data-val', default='data/val/', type=str,
                help='path to val data folder')
parser.add_argument('--ann-path-val', default='val.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('--rois-path-val', default='val.json',
                type=str, help='path to ROIs annotations in coco format')

parser.add_argument('--save-folder', default='TRAIN/new_test/',
                help='location to save checkpoint models')
parser.add_argument('--snapshot', default='boneage_bonet_weights.pth',
                help='path to weight snapshot file')
parser.add_argument('--optim-snapshot', type=str,
                default='boneage_bonet_optim.pth',
                help='path to optimizer state snapshot')

parser.add_argument('--eval-first', default=False, action='store_true',
                help='evaluate model weights before training')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                help='number of data loading workers (default: 4)')

_StoreAction(option_strings=['-j', '--workers'], dest='workers', nargs=None, const=None, default=4, type=<class 'int'>, choices=None, help='number of data loading workers (default: 4)', metavar='N')

In [10]:
# Training procedure settings
parser.add_argument('--batch-size', default=1, type=int,
                help='Batch size for training')
parser.add_argument('--epochs', type=int, default=20,
                help='upper epoch limit')
parser.add_argument('--lr', '--learning-rate', default=1e-5, type=float,
                help='initial learning rate')
parser.add_argument('--patience', default=2, type=int,
                help='patience epochs for LR decreasing')
parser.add_argument('--start-epoch', type=int, default=1,
                help='epoch number to resume')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--log-interval', type=int, default=30, metavar='N',
                    help='report interval')

parser.add_argument('--gpu', type=str, default='2,3')

_StoreAction(option_strings=['--gpu'], dest='gpu', nargs=None, const=None, default='2,3', type=<class 'str'>, choices=None, help=None, metavar=None)

In [11]:
args = parser.parse_args(args=[])
args

Namespace(ann_path_train='train.csv', ann_path_val='val.csv', batch_size=1, cropped=False, data_train='data/train/', data_val='data/val/', dataset='RSNA', epochs=20, eval_first=False, gpu='2,3', heatmaps=False, log_interval=30, lr=1e-05, optim_snapshot='boneage_bonet_optim.pth', patience=2, rois_path_train='train.json', rois_path_val='val.json', save_folder='TRAIN/new_test/', seed=1111, snapshot='boneage_bonet_weights.pth', start_epoch=1, workers=4)

In [12]:
args_dict = vars(args)
print('Argument list to program')
print('\n'.join(['--{0} {1}'.format(arg, args_dict[arg])
                 for arg in args_dict]))
print('\n\n')

Argument list to program
--heatmaps False
--cropped False
--dataset RSNA
--data_train data/train/
--ann_path_train train.csv
--rois_path_train train.json
--data_val data/val/
--ann_path_val val.csv
--rois_path_val val.json
--save_folder TRAIN/new_test/
--snapshot boneage_bonet_weights.pth
--optim_snapshot boneage_bonet_optim.pth
--eval_first False
--workers 4
--batch_size 1
--epochs 20
--lr 1e-05
--patience 2
--start_epoch 1
--seed 1111
--log_interval 30
--gpu 2,3





In [13]:
torch.manual_seed(args.seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

In [14]:
if not os.path.exists(args.save_folder):
    os.makedirs(args.save_folder)

In [15]:
# Horovod settings
hvd.init()
torch.cuda.set_device(hvd.local_rank())
torch.cuda.manual_seed(hvd.size())

args.distributed = hvd.size() > 1
args.rank = hvd.rank()
args.size = hvd.size()

In [16]:
# CREATE THE NETWORK ARCHITECTURE AND LOAD THE BEST MODEL
if args.heatmaps:
    from Bonet.models.bonet_heatmap import BoNet
else:
    from Bonet.models.bonet import BoNet

In [17]:
net = BoNet()

In [18]:
if args.rank == 0:
    print('---> Number of params: {}'.format(
        sum([p.data.nelement() for p in net.parameters()])))

---> Number of params: 123172057


In [19]:
if osp.exists(args.snapshot):
    model_to_load=args.snapshot
else:
    model_to_load=args.save_folder+'/'+args.snapshot

In [20]:
if osp.exists(model_to_load) and args.rank == 0:
    print('Loading state dict from: {0}'.format(model_to_load))
    snapshot_dict = torch.load(model_to_load, map_location=lambda storage, loc: storage)
    weights= net.state_dict()
    new_snapshot_dict=snapshot_dict.copy()
    for key in snapshot_dict:
        if key not in weights.keys():
            new_key='inception_v3.'+key
            new_snapshot_dict[new_key]=snapshot_dict[key]
            new_snapshot_dict.pop(key)

    net.load_state_dict(new_snapshot_dict)

net = net.to(device)

In [21]:
# Criterion
criterion = nn.L1Loss()

In [22]:
# Optimizer
optimizer = optim.Adam(net.parameters(), lr=args.lr * args.size)
annealing = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.8, patience=args.patience, cooldown=5,
    min_lr=0.00001, eps=0.00001, verbose=True)

In [23]:
if osp.exists(args.optim_snapshot):
    optim_to_load=args.optim_snapshot
else:
    optim_to_load=args.save_folder+'/'+args.optim_snapshot

In [24]:
if osp.exists(optim_to_load):
    print('loading optim snapshot from {}'.format(optim_to_load))
    optimizer.load_state_dict(torch.load(optim_to_load, map_location=lambda storage,
                                             loc: storage))

In [25]:
# Horovod
hvd.broadcast_parameters(net.state_dict(), root_rank=0)

optimizer = hvd.DistributedOptimizer(
    optimizer, named_parameters=net.named_parameters())
hvd.broadcast_optimizer_state(optimizer, root_rank=0)
group = optimizer.param_groups[0]
group['betas'] = (float(group['betas'][0]), float(group['betas'][1]))

In [26]:
# Dataloaders
train_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.RandomAffine(
                                   20, translate=(0.2, 0.2),
                                   scale=(1, 1.2)),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()])
train_transform

Compose(
    Resize(size=(500, 500), interpolation=bilinear, max_size=None, antialias=None)
    RandomAffine(degrees=[-20.0, 20.0], translate=(0.2, 0.2), scale=(1, 1.2))
    RandomHorizontalFlip(p=0.5)
    ToTensor()
)

In [27]:
val_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.ToTensor()])
val_transform

Compose(
    Resize(size=(500, 500), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
)

In [28]:
if args.heatmaps:
    from Bonet.data.data_loader import Boneage_HeatmapDataset as Dataset
else:
    from Bonet.data.data_loader import BoneageDataset as Dataset

In [29]:
train_dataset = Dataset(args.data_train, args.ann_path_train,args.rois_path_train,
                                   img_transform=train_transform,crop=args.cropped,dataset=args.dataset)
val_dataset = Dataset(args.data_val, args.ann_path_val,args.rois_path_val,
                                 img_transform=val_transform,crop=args.cropped,dataset=args.dataset)

FileNotFoundError: ignored